In [1]:
## Import the required libraries and read the data into memory

import duckdb
import pandas as pd
from plotnine import (
    ggplot,
    aes,
    geom_bar,
    theme,
    element_text
)

df = pd.read_csv('./data/Trade_DetailedTradeMatrix.csv', encoding='latin-1')

colnames = []
for i in df.columns:
    i = i.replace(" ", "_")
    colnames.append(i)

df.columns = colnames

In [31]:
## This function calculates the trade dependency of a country by year by total import value.
## It fetches the top 20 countries by import value that the selected country imports from

def trade_dependency(country, year):
        
    dataframe = duckdb.sql('''
    select Partner_Countries,
    sum(Value) /
        (select
            sum(Value)
            from df
            where Year = {year}
            and Element = 'Import Value'
            and Reporter_Countries = '{country}'
        ) as imp_prop
    from df
    where Year = {year}
    and Element = 'Import Value'
    and Reporter_Countries = '{country}'
    group by Partner_Countries
    having imp_prop > 0
    order by imp_prop desc;
    '''.format(year = year, country = country)).fetchdf().head(21)

    return dataframe

trade_dependency('Saudi Arabia', 2019)

,Partner_Countries,imp_prop
0,India,0.135775
1,Brazil,0.086315
2,United States of America,0.070394
3,United Arab Emirates,0.057677
4,Argentina,0.050924
5,Türkiye,0.035603
6,France,0.030245
7,Egypt,0.029604
8,Netherlands,0.027197
9,Germany,0.026609


In [2]:
def trade_dependants(country, year):

    top_imports = duckdb.sql('''
    select Reporter_Countries,
        sum(Value) as imports_uae
        from df
        where Year = {year}
        and Element = 'Import Value'
        and Partner_Countries = '{country}'
        group by Reporter_Countries
        having imports_uae > 0
        order by imports_uae desc;
        '''.format(year = year, country = country)).fetchdf().head(21)

    total_imports = duckdb.sql('''
    select Reporter_Countries,
        sum(Value) as import_value
        from df
        where Year = {year}
        and Element = 'Import Value'
        group by Reporter_Countries
        having import_value > 0
        order by import_value desc;
        '''.format(year = year)).fetchdf()

    dependants = duckdb.sql(
        '''
        drop table if exists imps;
        drop table if exists dependants;

        create table imps as
            select * from top_imports
            left join total_imports
            on top_imports.Reporter_Countries = total_imports.Reporter_Countries
            order by import_value desc;

            select Reporter_Countries,
                imports_uae / import_value as dep_ratio


                from imps
                where dep_ratio >= 0.04
                order by dep_ratio desc;
        ''').fetchdf()
    
    return dependants

trade_dependants('United Arab Emirates', 2021)

,Reporter_Countries,dep_ratio
0,Oman,0.350573
1,Maldives,0.203470
2,Afghanistan,0.186384
3,Bahrain,0.175980
4,Syrian Arab Republic,0.173320
5,Kuwait,0.155162
6,Libya,0.144555
7,Iran (Islamic Republic of),0.107553
8,Yemen,0.101324
9,Benin,0.091926
